# Local Auto categorization
<hr>
<div style="background: white"><img src="architecture.png" style="width:50%"/></div>
Locally it is possible to do auto-categorization. To do this it is required to collect data, train the model (optionally validate the model) and use the model to predict categories to any given arbitrary text. As the pictures shows the application consists of two parts, one part is for training an auto-categorization model the other part is a web service that can be hosted anywhere to serve the auto-categorization to any client.

## Collect data
To collect the data it is required to set up some environment variables located in .env.sample.

In [1]:
import sys, os
sys.path.insert(0, '../')
%cat ../.env.sample

ENV=production

CS_TOKEN=
CS_URL=

DB_URL=soldr-dev.cdawc3jitldx.eu-west-1.redshift.amazonaws.com
DB_USER=
DB_PASSWORD=
DB_PORT=5439
DB_NAME=


### Fetching data from Mittmedia article database
The important part about this section of the process is to save a json file in the right format to be used in next steps of training the model. The format should be in the following way:
```
{
  "articles": [
    {
      "categories": ["Sport", "Ekonomi",...],
      "category_ids": [1,2,....],
      "text": "Text to categorize...",
      "lead": null,
      "headline": null
    }
  ]
}
```

In [2]:
document_data = '../learning/data/notebook_mm_articles.json'

In [3]:
from learning.mm_services import fetch_data
# fetch_data.main(document_data)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [4]:
{'Väder': 4459, 'Miljö': 1908, 'Livsstil & fritid': 3727, 'Politik': 7076, 'Ekonomi, näringsliv & finans': 17327, 'Kultur & nöje': 6256, 'Vetenskap & teknologi': 525, 'Olyckor & katastrofer': 5169, 'Personligt': 3769, 'Skola & utbildning': 2284, 'Brott & straff': 9221, 'Samhälle & välfärd': 7787, 'Arbetsmarknad': 1424, 'Hälsa & sjukvård': 2752, 'Sport': 21551}

{'Väder': 4459,
 'Miljö': 1908,
 'Livsstil & fritid': 3727,
 'Politik': 7076,
 'Ekonomi, näringsliv & finans': 17327,
 'Kultur & nöje': 6256,
 'Vetenskap & teknologi': 525,
 'Olyckor & katastrofer': 5169,
 'Personligt': 3769,
 'Skola & utbildning': 2284,
 'Brott & straff': 9221,
 'Samhälle & välfärd': 7787,
 'Arbetsmarknad': 1424,
 'Hälsa & sjukvård': 2752,
 'Sport': 21551}

## Train the model
This section will show all steps to take to train the model for auto-categorization (e.g. configuration and training).

### Configuration
To be able to train the model some configuration has to be made. Deciding where the models are stored and where training data is read among other model hyper parameters can be tweaked in a yaml file in `config/` folder. For demonstration purposes we will overwrite these settings in the notebook to enable full cusomization while experimenting.

In [5]:
os.environ['ENV'] = '../notebook-model-config'
import learning.config

In [6]:
learning.config.model = {
    'path': '../learning/trained-models/',
    'vec_model': {
        'name': 'gensim_models/word2vec_MM_new_category_tree.model',
        'type': 'word2vec',
        'train': True
    },
    'categorization_model': {
        'name': 'lstm-multi-categorizer-new-category-tree.model',
        'type': 'blstm',
        'model_checkpoint': False
    }
}
learning.config.data = {
    'path': os.path.dirname(document_data) + '/',
    'articles': os.path.basename(document_data),
    'target_categories': 'new_top_categories.txt',
    'stop_words': 'stop_words.txt'
}
learning.config.verbose = True

Training the model is done with a simple function call to the module. In the background this function will train word/document vectors (if specified in configuration). The function will then filter out articles from the fetched ones such that we have equal amount of articles per category label before mapping the texts to input-vectors.

In [7]:
from learning import model

In [8]:
# model.train_and_store_model()

## Use the trained model
Now that we have the trained model we can use it in an practical example by first load the model from disk and then get a text from anywhere to predict a category on.

In [9]:
input_file = learning.config.model['categorization_model']['name']
predictor = model.BLSTMCategorizer(input_file, '../learning/trained-models/lstm-multi-categorizer-new-category-tree.model')





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Loaded model with Allsvenskan,Trav,Elitettan,Division 6 herr,Livsstil & fritid,Fotboll,Bilsport,Hockeyettan,Elitserien herr,Bowling,Sport,Skidåkning,Division 1,Division 2 dam,Innebandy,Division 1 herr,Sporter,SHL,Superettan,SDHL,Division 2 herr,Elitserien,Hockeytrean,Division 3 dam,Speedway,Hockeyallsvenskan,Division 1 dam,Basket,Bandy,Division 5 herr,Friidrott,Ishockey,Handboll,Södra Norrland,Division 4 herr,Motorcykelsport,Längdskidåkning,Hockeytvåan,Norra,Mellersta Norrland,Division 3 herr categories


In [10]:
text = "De allsvenska tränarna och även en hel del spelarprofiler samlades på Idrottens hus i Stockholm för en första historisk upptaktsträff. Och mycket fokus hamnade naturligtvis på frågan om vilket lag som förväntas ha bäst chanser att vinna serien och ta direktplatsen till elitserien. Men i det traditionsenliga förhandstipset var tränarna inte särskilt överens. Tre lag – Ljusdal, Nässjö och Örebro trodde på sig själva. Och det är också en trio som med stor sannolikhet kommer att vara med i toppen av tabellen. Nässjö fick flest röster – sex stycken. Och tränaren Patrik Gustafsson Thulin fick också frågan hur det kändes att pekas ut som favoriter. – Det är jättekul. Det är ett kvitto på att vi gör många saker som är bra och som motståndarlagen är jobbiga. Då är vi på rätt väg. Föreningen förbereder sig också att vi ska vara redo om vi tar steget upp, säger han. LÄS MER: Vi sände den historiska upptaktsträffen för allsvenskan – se reprisen Det snackades också rätt mycket om Ljusdal (fyra röster) som spelade senast i elitserien säsongen 2013/14. Nu hoppas allsvenskans nordligaste klubb på att ta ännu ett kliv. – Vi har en tydlig målsättning vad vi vill och det är att ta oss tillbaka till elitserien. Jag tycker det känns bra. Det är kanske lite logiskt hur historien hur det har varit de senaste åren. Vi var väldigt nära förra säsongen. Ofta räcker det med åtta poäng i en kvalserie, men inte då, säger tränaren Patrik Larsson. Falun fick tre röster, Örebro två och Gripen en. – Klubben har inget uttalat mål om elitserien. Men vi jobbar på alla fronter för det. Vi har varit i två kval under de tre senaste säsonger och vi har lärt oss mycket av. Vi vet inte så mycket om söderlagen. Men Nässjö ser starka ut både på papperet och resultatmässig under försäsongen, säger Niclas Groth, tränare tillsammans med Joakim Forslund i Falu BS. Bandypuls har tidigare berättat om det nya serieupplägget. De är hela 16 lag som får plats i den första upplagan av superallsvenskan som består av två pooler – en med åtta söderlag och en med två norrlag. I respektive pool möts alla lag två gånger och därefter blir det enkelmöten med lag från andra poolen. Ettan spelar i elitserien nästa säsong och tvåan och trean får kvala."
predictor.categorize_text([text])

[('Division 5 herr', 1.1749807526939549e-06),
 ('Division 6 herr', 1.5709370018157642e-06),
 ('Hockeytrean', 2.2633578282693634e-06),
 ('Division 2 dam', 4.693012215284398e-06),
 ('Division 3 dam', 5.870311269973172e-06),
 ('Trav', 8.229661034420133e-06),
 ('Bowling', 8.985126441984903e-06),
 ('Hockeytvåan', 1.2494234397308901e-05),
 ('Friidrott', 2.3745054932078347e-05),
 ('Mellersta Norrland', 3.018754614458885e-05),
 ('Division 1', 4.6757166273891926e-05),
 ('Elitettan', 5.294038783176802e-05),
 ('Basket', 5.334807065082714e-05),
 ('SDHL', 5.77784885535948e-05),
 ('Bilsport', 9.30681053432636e-05),
 ('Handboll', 0.00011511090269777924),
 ('Hockeyettan', 0.0002134857204509899),
 ('Längdskidåkning', 0.00022847694344818592),
 ('Division 4 herr', 0.0002521077112760395),
 ('Livsstil & fritid', 0.0002879078092519194),
 ('Innebandy', 0.0002941703423857689),
 ('Division 1 dam', 0.0002958025725092739),
 ('Division 2 herr', 0.0004995414055883884),
 ('Skidåkning', 0.000755502434913069),
 ('Söd

### Use the trained model 2
Another way to do it is by starting the server and do a request

In [11]:
%%capture
import web.app
import multiprocessing
import requests
app_process = multiprocessing.Process(target=lambda: web.app.app.run(host='0.0.0.0', port=8080))

In [12]:
app_process.start()

 * Serving Flask app "web.app" (lazy loading)
 * Debug mode: off
 * Environment: production
   Use a production WSGI server instead.


 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Oct/2019 13:48:02] "GET /ping HTTP/1.1" 200 -


In [13]:
requests.get('http://localhost:8080/ping').text

'\n'

Then do a request to the app with something like the following structure

In [ ]:
document = {
    'body': text,
    'categories2': None,
    'uuid': None
}
url = 'http://localhost:8080/invocations'
requests.post(url, json=document).text

### Stop the server

In [ ]:
# app_process.terminate()
# app_process.join()